In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import(
    when, col, substring, expr, round,
    to_date, year, sum, count, lit,
    regexp_replace, lpad, length,
    month, count, avg
)
spark = SparkSession.builder.appName("agrupar-sumarizar") \
    .config("spark.sql.legacy.timeParserPolicy","LEGACY") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.sql.repl.eagerEval.maxNumRows", 20) \
    .getOrCreate()
df = spark.read.csv("curso-analise-de-dados/dados/sim_salvador_2023.csv",
                    header=True,
                    inferSchema=True
)
df.show(5)

+---------+--------+-----+----+-------+--------+--------+
|CODMUNRES| DTOBITO|IDADE|SEXO|RACACOR|CAUSABAS|TIPOBITO|
+---------+--------+-----+----+-------+--------+--------+
|   292720| 2012023|  457|   2|      4|    J980|       2|
|   292720| 5012023|  482|   2|      2|    I500|       2|
|   292720| 7012023|  452|   2|      4|    C220|       2|
|   292720| 8012023|  456|   2|      4|    W789|       2|
|   292720|10012023|  468|   1|      4|    X954|       2|
+---------+--------+-----+----+-------+--------+--------+
only showing top 5 rows



In [60]:
# Resumo funcional Esse código:
# Corrige o formato da data de óbito (DTOBITO).
# Converte o campo para um tipo de data válido.
# Cria uma coluna derivada com o ano do óbito, facilitando análises temporais.
# DTOBITO com date e ano_obito
df = df.withColumn(
    "DTOBITO_clean", 
    #Converte a coluna DTOBITO para string.
    # Garante que ela tenha exatamente 8 caracteres, preenchendo com zeros à esquerda quando necessário.
    lpad(col("DTOBITO").cast("string"), 8, "0")
).withColumn(
    # data no formato ddMMyyyy precisam ter 8 dígitos.
    # Exemplo:
    # 1012023 → 01012023
    # 1122022 → 01122022
    # conversão para o tipo date
    "DTOBITO_dt", 
    # Converte a coluna DTOBITO_clean de string para o tipo Date do Spark.
    # Usa explicitamente o formato ddMMyyyy.
    # DTOBITO_dt passa a ser uma data válida, permitindo operações temporais (filtros, agrupamentos, etc.).
    to_date(col("DTOBITO_clean"), "ddMMyyyy")
).withColumn(
    # Extrai o ano da data de óbito.
    # Cria a coluna ano_obito como um inteiro.
    # Exemplo:
    # 2023-01-01 → 2023
    "ano_obito", 
    year(col("DTOBITO_dt"))
).withColumn(
    "mes_obito",
    month(col("DTOBITO_dt"))
)


In [61]:
# Media da idade dos obitos_totais
# Extração do tipo de idade
# Converte IDADE para string
# Extrai o primeiro caractere
# Cria a coluna tipo_idade
# Resultado: Identifica em qual unidade a idade está expressa (anos, meses etc.)
df = df.withColumn(
    "tipo_idade",
    substring(col("IDADE").cast("string"), 1,1)
).withColumn(
    # Extração do valor da idade
    # Extrai a partir do segundo caractere 100 garante que todo o restante da string seja capturado
    # Cria a coluna idade 
    # Resultado: Valor bruto da idade, ainda como string
    "idade_raw",
    substring(col("IDADE").cast("string"), 2, 100) 
)

In [62]:
# 3. Padronização da idade para ANOS
df = df.withColumn(
    "idade_anos",
    when(col("tipo_idade") == "4", col("idade_raw").cast("int"))        # anos
    .when(col("tipo_idade") == "3", col("idade_raw").cast("int") / 12)  # meses
    .when(col("tipo_idade") == "2", col("idade_raw").cast("int") / 365) # dias
    .when(col("tipo_idade") == "1", col("idade_raw").cast("int") / 8760)# horas
)
# 4. Agregação: óbitos por mês + média de idade
df_obitos_por_mes = df \
    .filter(col("DTOBITO_dt").isNotNull()) \
    .groupBy("ano_obito", "mes_obito") \
    .agg(
        count("*").alias("obitos_totais"),
        avg("idade_anos").alias("idade_media_anos")
    ) \
    .orderBy("ano_obito", "mes_obito")
df_obitos_por_mes = df_obitos_por_mes.withColumn(
    "idade_media_anos",
    round(col("idade_media_anos"), 2)
)
df_obitos_por_mes.show(truncate=False)
#---------------------------------------------#
# O que esse pipeline faz (resumo funcional)
# Corrige e converte DTOBITO para data válida
# Extrai ano e mês do óbito
# Decodifica a variável IDADE do DATASUS
# Converte todas as idades para anos
# Calcula, por ano/mês:
# Total de óbitos
# Idade média dos óbitos
#---------------------------------------------#

+---------+---------+-------------+----------------+
|ano_obito|mes_obito|obitos_totais|idade_media_anos|
+---------+---------+-------------+----------------+
|2023     |1        |23           |69.3            |
|2023     |2        |15           |70.33           |
|2023     |3        |14           |59.86           |
|2023     |4        |22           |55.73           |
|2023     |5        |30           |63.12           |
|2023     |6        |25           |64.48           |
|2023     |7        |20           |71.35           |
|2023     |8        |20           |68.25           |
|2023     |9        |22           |68.73           |
|2023     |10       |20           |69.95           |
|2023     |11       |19           |71.24           |
|2023     |12       |31           |66.94           |
+---------+---------+-------------+----------------+

